In [ ]:
import pandas as pd
import json
from sqlalchemy import create_engine

db_url = "mysql+pymysql://root:@localhost/keranjang_homemade"
engine = create_engine(db_url)

query = """
SELECT 
    DATE_FORMAT(o.order_date, '%%M') AS month,  -- Convert to month name
    SUM(CASE WHEN o.order_type_id = 1 THEN oi.quantity * oi.price ELSE 0 END) AS online_sales,
    SUM(CASE WHEN o.order_type_id = 2 THEN oi.quantity * oi.price ELSE 0 END) AS offline_sales
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY month
ORDER BY MIN(o.order_date);  -- Ensure correct order by the actual month
"""

df = pd.read_sql(query, engine)

df = df[(df['online_sales'] > 0) | (df['offline_sales'] > 0)]  # Hanya ambil bulan yang ada penjualannya

data = {
    "labels": df["month"].tolist(),
    "online_sales": df["online_sales"].tolist(),
    "offline_sales": df["offline_sales"].tolist()
}

with open('C:\\laragon\\www\\keranjang_homemade\\sales_data.json', 'w') as f:
    json.dump(data, f)